In [1]:
import os
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import torch
import torch.nn.functional as F

In [2]:
torch.manual_seed(1234)
node = 100
x = torch.zeros(1,node+1)
x[0,0] = 100
adj = torch.sign(F.relu(torch.tril(torch.triu(torch.bernoulli(torch.tensor(0.8).repeat(x.shape[1],x.shape[1])), -3), 3)-torch.eye(x.shape[1])))+torch.eye(x.shape[1])
adj[-1,:] = torch.zeros_like(adj[-1,:])
adj[-1,-1] =1 
#adj = torch.tril(torch.triu(torch.ones(x.shape[1],x.shape[1])),3)
x

def stoch_step(x, adj, c = 1e-3):
    #m = torch.cdist(x.transpose(-2,-1).float(), x.transpose(-2,-1).float())/x.shape[1]**0.5
    k = torch.tensor([range(1, x.shape[-1]+1)]).transpose(-2,-1)*torch.tensor([1/x**2 for x in range(1,x.shape[-1]+1)])
    m = -c*(x.float()-x.transpose(-2,-1).float())*k/x.shape[1]**0.5
    m = F.softmax(m.float().masked_fill(adj == 0., float("-infinity")), 1)
    index, freq = m[x[0]>0.,:].multinomial(1).reshape(-1).unique(return_counts = True)
    
    x[0,x[0]>0.] -= 1
    x[0,index] += freq

    # this means that every 180 second I expect a new person at the entrance
    if np.random.rand()< 1/180:
        x[0,0]+=1
    return x

persons = torch.sum(x).item()
evolution = [x.clone()]
cont = True
step = 1
change = True
while (evolution[-1][0,-1] != torch.sum(evolution[-1]).item()) & cont:
    x = stoch_step(x, 
                   adj, 
                   c = 1e2)
    evolution.append(x.clone())
    step += 1
    
    if change & (step > 3600):
        adj = torch.triu(adj-torch.eye(x.shape[1]))
        adj[-1,-1] =1 
        change = False
    
    if step >10000:
        cont = False
evolution = torch.vstack(evolution)
d = {}
for i in range(x.shape[1]):
    d[f"node {i}"] = evolution[:,i].numpy()

nodes = [f"node {i}" for i in range(0, node, 10)]
nodes.append(f"node {node}")
px.line(d,
        y = nodes,
        title = "People flow", 
        height = 600, 
        width = 1200).update_layout(xaxis_title = "step", yaxis_title =  "people")

KeyboardInterrupt: 

In [ ]:
os.system("gnome-terminal -e 'bash -c \"cd /home/andrea/Scrivania/Tesi/leonardo; jupyter-lab; exec bash\"'")
os.system("gnome-terminal -e 'bash -c \"watch -n 1 sensors; exec bash\"'")
os.system("gnome-terminal -e 'bash -c \"htop; exec bash\"'")
os.system("gnome-terminal -e 'bash -c \"nvtop; exec bash\"'")